# Test du modèle de reconnaissance faciale sur une image

Ce notebook permet de charger une image, détecter les visages, et prédire les identités à l'aide d'un modèle CNN préalablement entraîné.

In [1]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

# Constantes
IMG_SIZE = 200
image_path = 'test2.jpeg'  

# Chargement du modèle et de l'encodeur
model = load_model('model_faces.h5')
with open('label_encoder.pkl', 'rb') as f:
    le = pickle.load(f)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Chargement et traitement de l'image
image = cv2.imread(image_path)
if image is None:
    raise ValueError(f"[ERREUR] Impossible de charger l'image à {image_path}")
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))
print(f"[INFO] {len(faces)} visage(s) détecté(s).")

# Boucle sur chaque visage détecté
for (x, y, w, h) in faces:
    face_roi = image[y:y+h, x:x+w]
    face_gray = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
    face_resized = cv2.resize(face_gray, (IMG_SIZE, IMG_SIZE))
    face_normalized = face_resized.astype("float") / 255.0
    face_reshaped = np.reshape(face_normalized, (1, IMG_SIZE, IMG_SIZE, 1))
    
    prediction = model.predict(face_reshaped)
    class_index = np.argmax(prediction[0])
    confidence = prediction[0][class_index]
    predicted_name = le.inverse_transform([class_index])[0]
    label = f"{predicted_name} ({confidence*100:.1f}%)"
    print(f"[INFO] Prédiction : {label}")
    
    color = (0, 255, 0) if confidence > 0.75 else (0, 255, 255)
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

# Affichage du résultat final
cv2.imshow("Reconnaissance Faciale", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

KeyboardInterrupt: 